## Using ChatGPT to interact with Warren Buffett's letters to shareholders



![Warren Buffet](data/buffett_smaller.png) 

In [1]:
import os
import ast
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from config import *

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
# Let's load the letters to shareholders

loader = DirectoryLoader('data/')
docs = loader.load()

In [3]:
# Setting the embeddings model

embeddings = OpenAIEmbeddings(model=EMBEDDINGS_MODEL)

In [4]:
# Let's split all the letters to shareholders into separate files using the simple RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap  = 20,
    length_function = len,
)

texts = []
metadatas = []
for single_doc in docs:
    for single_part in text_splitter.split_text(single_doc.page_content):
        # Storing all split docs in single variable
        texts += [single_part]
        # Metadata is saved in order to understand which letters the answer was synthesized from
        metadatas += [ast.literal_eval(single_doc.json())['metadata']]


In [5]:
# Let's generate embeddings and store them in the chromadb
docsearch = Chroma.from_texts(texts, embeddings, metadatas=metadatas)

Using embedded DuckDB without persistence: data will be transient


In [ ]:
# Let's use the basic load_qa_with_sources_chain chain

def answer_question_from_letters(query, k_docs_to_generate_answer_from = K_DOCS_TO_GENERATE_ANSWER_FROM):
    docs = docsearch.similarity_search(query, k = k_docs_to_generate_answer_from)
    chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine")
    
    return chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [7]:
query = "What does Warren Buffett think about risks?"

In [8]:
result = answer_question_from_letters(query)

In [9]:
print(result)

{'output_text': '\n\nWarren Buffet is known for his conservative approach to investing and his aversion to taking risks. He has famously said, "Risk comes from not knowing what you\'re doing," and he has been quoted as saying, "I never attempt to make money on the stock market. I buy on the assumption that they could close the market the next day and not reopen it for five years." He also applies this same conservative approach to the insurance business, avoiding risks that are inappropriate in relation to the resources of Berkshire. In a speech given on March 26, 1979, he stated, "We will reject interesting opportunities rather than over-stretch ourselves. We will attempt to be alert to possibilities of attractive risk/reward ratios, but we will not bend our general rules in an effort to make an extra point or two." On February 28, 1989, he further emphasized the importance of being aware of the risks associated with any investment, noting in a statement to the board of directors on M

In [10]:
query = "How does Warren Buffett handle CEOs of the companies he acquires?"


In [11]:
result = answer_question_from_letters(query)

In [12]:
print(result)

{'output_text': "\n\nWarren Buffett is known for taking a hands-off approach when it comes to the CEOs of the companies he acquires. In 1979, Buffett Partnership, Ltd. acquired control of the company, and since then, Buffett has typically allowed the existing management team to remain in place and continue running the business as they have been, even if the CEO of the acquirer is replaced. He will provide guidance and advice when needed, drawing on his decades of experience as Chairman of the Board since February 28, 1989 and his control of Berkshire since 2006, and will evaluate the CEO to ensure they are the right fit for the company. He will look for business savvy, a competence that is far from commonplace, and will assess the CEO's compensation to ensure it is appropriate and reasonable. Additionally, he will evaluate proposed acquisitions to determine if they are likely to create or destroy per-share value, and will help the CEO in evaluating acquisitions. Of all their activities